***INDICE***
1. Caricamento file
2. Caricamento dei testi dei file in oggetti Document()
3. Set up del LLM
4. Creazione degli indici
5. Caricamento indici da disco
6. Query semplice sull'indice

**INSTALLAZIONE DELLE LIBRERIE NECESSARIE**
- *Le librerie sono state installate in un env di anaconda* 

*Librerie necessarie*
- !pip install llama-index 
- !pip install openai 
- !pip install langchain 
- !pip install gpt-index 
- !pip install PyPDF2 
- !pip install pypdf

**1. CARICAMENTO FILE**

In [3]:
import os
files = os.listdir("./documenti")          # otteniamo la lista di tutti i file contenuti nella directory di lavoro di colab

lista_pdf = []
for file in files:              # col ciclo otterremo la lista "lista_pdf" che conterrà i file in formato 
    if ".pdf" in file:          # pdf presenti nella direrctory
        lista_pdf.append(file)

print(lista_pdf)

['Chain of thoughts.pdf', 'Prompt patterns.pdf']


**2. CARICAMENTO DEI TESTI DEI FILE IN OGGETTI DOCUMENT()**

In [19]:
from llama_index.readers import SimpleDirectoryReader     # importiamo SimpleDirectoryReader che ci permetterà di leggere il file ceh desideriamo


doc_set = {}              # insieme che conterrà coppie "chiavi:valori" -> nome del file : [Document(text = '')]     perr ogni file caricato
all_docs = []             # Lista che conterrà i testi di ciascun file come elementi

i = 0

for nomefile in lista_pdf:
  # "input_dir" -> innseriamo la directory "documenti" che contiene i nostri pdf
  # "required_exts": imposta l'estensione di cui abbiamo bisogno (anche qui deve essere una lista 
  
  pdf_docs = SimpleDirectoryReader(input_dir = "./documenti/", required_exts = [".pdf"]).load_data()        # pdf_docs -> variabile che contiene il testo del documento letto da SimpleDirectoryReader
  
  i+=1
  # insert nomefile metadata into each file
  for d in pdf_docs:
    d.extra_info = {"nome file": nomefile}            # inserendo il nome del file come metadato permetterà un'interrogazione più semplice
    
  doc_set[nomefile] = pdf_docs
  all_docs.extend(pdf_docs)

In [20]:
print(doc_set)

{'Chain of thoughts.pdf': [Document(text='Chain-of-Thought Prompting Elicits Reasoning\nin Large Language Models\nJason Wei Xuezhi Wang Dale Schuurmans Maarten Bosma\nBrian Ichter Fei Xia Ed H. Chi Quoc V . Le Denny Zhou\nGoogle Research, Brain Team\n{jasonwei,dennyzhou}@google.com\nAbstract\nWe explore how generating a chain of thought —a series of intermediate reasoning\nsteps—signiﬁcantly improves the ability of large language models to perform\ncomplex reasoning. In particular, we show how such reasoning abilities emerge\nnaturally in sufﬁciently large language models via a simple method called chain-of-\nthought prompting , where a few chain of thought demonstrations are provided as\nexemplars in prompting.\nExperiments on three large language models show that chain-of-thought prompting\nimproves performance on a range of arithmetic, commonsense, and symbolic\nreasoning tasks. The empirical gains can be striking. For instance, prompting a\nPaLM 540B with just eight chain-of-though

**3. SET UP DEL LLM**

In [21]:
import os

# importiamo la chiave che sarà necessaria per impostare il LLM da usare, la nostra è una chiave di OpenAI
os.environ['OPENAI_API_KEY'] = 'sk-...Iy'     

''' provare ad usare MockLLMPredictor per ottenere il numero di token usati -> from llama_index import MockLLMPredictor''' 

# Set up del LLM
from llama_index import LLMPredictor, GPTVectorStoreIndex, PromptHelper, GPTListIndex, ServiceContext 
from langchain.chat_models import ChatOpenAI

# Defining LLm
llm_predictor = LLMPredictor(llm = ChatOpenAI(temperature = 0, model_name = 'gpt-3.5-turbo', openai_api_key= os.environ["OPENAI_API_KEY"]))


# Define PromptHelper
## Set Maximum input size
max_input_size = 4096

## Set max number of output tokens
num_output = 256

## Sex maximum chunk overlap
max_chunk_overlap = 20


prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)       # salviamo le impostazioni per il prompting del modello



service_context = ServiceContext.from_defaults(llm_predictor = llm_predictor,     # impostiamo il contesto di rferimento, ovvero il modello e le impostazioni
                                               prompt_helper = prompt_helper)     # per il suo prompting

**4. CREAZIONE DEGLI INDICI**

In [22]:
# initialize simple vector indices + global vector index


index_set = {}                    # questo insieme conterrà coppie chiavi:valori nella forma -> "nomefile": indice
for nomefile in lista_pdf:
    
  # Creazione di un indice per ogni documento
  cur_index = GPTVectorStoreIndex.from_documents(doc_set[nomefile], service_context=service_context)  # GPTVectorStoreIndex creerà l'indice basandosi sull'oggetto Document(text ='') del file selezionato

  index_set[nomefile] = cur_index                                                                     # Carichiamo nell'insieme la chiave "nomefile" e il relativo valore "cur_index"

  # Salviamo ogni indice come file .json
  cur_index.storage_context.persist(persist_dir = f'''./storage/index_{nomefile.replace(".pdf","")}.json''')

**5. CARICAMENTO INDICI DA DISCO**

In [23]:
from llama_index import StorageContext, load_index_from_storage


# Load indices from disk
index_set = {}
for nomefile in lista_pdf:
  # rebuild storage context
  storage_context = StorageContext.from_defaults(persist_dir = f'''./storage/index_{nomefile.replace(".pdf","")}.json/''')

  # load index
  cur_index = load_index_from_storage(storage_context)

  ###cur_index = GPTVectorStoreIndex.load_from_disk(f'index_{nomefile.replace(".pdf","")}.json')  # # non viene più usato
  index_set[nomefile] = cur_index
print(index_set)

{'Chain of thoughts.pdf': <llama_index.indices.vector_store.base.GPTVectorStoreIndex object at 0x000002035E309390>, 'Prompt patterns.pdf': <llama_index.indices.vector_store.base.GPTVectorStoreIndex object at 0x000002035E313790>}


**6. QUERY SEMPLICE SULL'INDICE**

In [24]:
# Nel caso avessimo un solo indice, possiamo provare a fare una query diretta sulla stesso, con impostazioni di default

query_engine = cur_index.as_query_engine()      # costruiamo sull'indice la query_engine che permetterà di interrogare l'indice

In [ ]:
response = query_engine.query("INSERISCI LA TUA QUERY")
print(response)

In [25]:
# Query sul file Chain of thoughts

response = query_engine.query("Please, consider the context given, thus the file Chain of thoughts.pdf. I would like you to explain it in 5 steps")
print(response)


1. Chain-of-thought prompting is a technique used to facilitate reasoning in language models.
2. It involves providing demonstrations of chain-of-thought reasoning in the exemplars for few-shot prompting.
3. The goal is to endow language models with the ability to generate a coherent series of intermediate reasoning steps that lead to the final answer for a problem.
4. Empirical evaluations on arithmetic, commonsense, and symbolic reasoning benchmarks show that chain-of-thought prompting outperforms standard prompting.
5. Chain-of-thought prompting has several attractive properties, such as allowing models to decompose multi-step problems into intermediate steps, providing an interpretable window into the behavior of the model, and being applicable to any task that humans can solve via language.


In [26]:
# Query sul file Chain of thoughts

response = query_engine.query("Please, consider the context given, thus the file Prompt Patterns.pdf. I would like you to explain it in 5 steps")
print(response)


1) The Prompt Patterns.pdf file provides a set of patterns that can be used to help users interact with a language learning machine (LLM). 

2) The Recipe Pattern helps users provide a sequence of steps to achieve a stated goal, given some partially provided “ingredients”. 

3) The Refusal Pattern helps users generate alternate questions when the LLM refuses to answer a question. 

4) The Context Manager Pattern enables users to specify or remove context for a conversation with an LLM, in order to focus the conversation on specific topics or exclude unrelated topics from consideration. 

5) The key ideas of the Context Manager Pattern involve providing explicit contextual statements about what to consider or ignore, in order to help the LLM better understand the question and generate more accurate responses.
